<a href="https://colab.research.google.com/github/Lilian0812/ai-interview-coach/blob/main/AI_Interview_Coach_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI 模擬面試官互動系統

有些人準備面試時會不知道該怎麼回答問題，也不知道自己的回答到底好不好，所以我設計了一個可以幫助大家練習面試的系統，讓使用者能在沒有真人面試官的情況下，透過線上互動練習面試問題，獲得具體的回饋和自我改善建議。

#1. 讀入Groq金鑰

In [ ]:
import os
from google.colab import userdata

In [ ]:
api_key = userdata.get('Groq')
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

In [ ]:
os.environ['OPENAI_API_KEY']=api_key

#2. prompt設定

定義面試問題: 指示 AI 生成具體且全中文的模擬面試問題

In [ ]:
system_prompt = """
你是一位專業的模擬面試官，根據使用者輸入的背景資訊（應徵職缺、自我介紹、專案經驗、技能）來針對職位與背景設計的面試問題，必須以全中文台灣話回應。
問題要具體、真實且與該職位有高度相關，請不要加入英文開場或備註提示。
"""

定義面試回饋方向: 以條列式呈現優點、缺點，並給予評分

In [ ]:
feedback_prompt_template = """
你是一位專業的面試官。以下是應徵者的回答：
"{answer}"

請你用全中文台灣話針對這個回答提供三項內容(不要使用英文)：
1. 優點（語言表達、邏輯、內容等）
2. 缺點或不足之處（可改善的地方，並給予具體建議）
3. 給予評分（邏輯性、完整度、說服力、專業性、案例具體性，每項滿分10分，在打完分數後要用文字敘述為什麼）

請條列式回答，並在最後做整體建議，需要五行以上，不要再寫跟分數相關的內容，可以給予一些鼓勵。或是直接提供範例回答，讓使用者清楚要如何改進。
"""

In [ ]:
context = {"background": "", "question": "", "history": []}

#3. 套件安裝

In [ ]:
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.2 MB/s eta 0:00:00


#4. 產生面試問題

根據使用者提供的背景資訊，產生具體的面試問題

In [ ]:
def generate_question(position, intro, skills, project):
    context["background"] = f"職位：{position}\n自我介紹：{intro}\n技能：{skills}\n專案經驗：{project}"
    prompt = f"{system_prompt}\n以下是使用者背景：\n{context['background']}"
    response = requests.post(
        f"{base_url}/chat/completions",
        headers={"Authorization": f"Bearer {api_key}"},
        json={
            "model": model,
            "messages": [
                {"role": "system", "content": "你是一位專業的模擬面試官"},
                {"role": "user", "content": prompt}
            ]
        }
    )
    question = response.json()["choices"][0]["message"]["content"]
    context["question"] = question
    return question

#5. 回饋

根據使用者對面試問題的回答，給予 AI 面試的具體回饋與評分

In [ ]:
def give_feedback(answer):
    prompt = feedback_prompt_template.format(answer=answer)
    response = requests.post(
        f"{base_url}/chat/completions",
        headers={"Authorization": f"Bearer {api_key}"},
        json={
            "model": model,
            "messages": [
                {"role": "system", "content": "你是一位專業的面試官"},
                {"role": "user", "content": prompt}
            ]
        }
    )
    feedback = response.json()["choices"][0]["message"]["content"]
    context["history"].append({
        "question": context["question"],
        "answer": answer,
        "feedback": feedback
    })

    return feedback

#6. 用 Gradio 打造 Web App

In [ ]:
import gradio as gr
import requests

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# AI 模擬面試官互動系統")

    with gr.Row():
        position = gr.Textbox(label="應徵職位")
        intro = gr.Textbox(label="自我介紹")
    with gr.Row():
        skills = gr.Textbox(label="技能")
        project = gr.Textbox(label="專案經驗")

    question_output = gr.Textbox(label="AI 生成的面試問題", lines=2)
    generate_btn = gr.Button("📌 生成面試問題")

    answer_input = gr.Textbox(label="你的回答", lines=4)
    feedback_output = gr.Textbox(label="AI 給你的回饋", lines=8)
    feedback_btn = gr.Button("📋 送出回答並獲得回饋")

    generate_btn.click(generate_question, inputs=[position, intro, skills, project], outputs=question_output)
    feedback_btn.click(give_feedback, inputs=answer_input, outputs=feedback_output)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6b5fc8fac6d17504eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6b5fc8fac6d17504eb.gradio.live
